In [30]:
import pandas as pd
import json
import cv2 as cv
import os
import glob

### Create shapes and images list file

In [50]:
img_dims = dict()
for imgfile in glob.glob('./images/*.jpg'):
    # print(imgfile) 
    img = cv.imread(imgfile)
    box_h, box_w, _ = img.shape
    img_dims[os.path.split(imgfile)[1]] = (box_w, box_h)
    

./images\bp1.jpg
./images\bp10.jpg
./images\bp11.jpg
./images\bp12.jpg
./images\bp13.jpg
./images\bp14.jpg
./images\bp15.jpg
./images\bp16.jpg
./images\bp17.jpg
./images\bp18.jpg
./images\bp19.jpg
./images\bp2.jpg
./images\bp20.jpg
./images\bp21.jpg
./images\bp22.jpg
./images\bp23.jpg
./images\bp24.jpg
./images\bp25.jpg
./images\bp3.jpg
./images\bp4.jpg
./images\bp5.jpg
./images\bp6.jpg
./images\bp7.jpg
./images\bp8.jpg
./images\bp9.jpg


In [41]:
output_dir = './YoloV3/data/blackpink/'
proj_name = 'blackpink'

In [56]:
shapes_file = os.path.join(output_dir, proj_name+'.shapes')
txt_file = os.path.join(output_dir, proj_name+'.txt')
img_file_prefix = './data/blackpink/images/'

# Create blackpink.shapes file
with open(shapes_file, "w") as shape_fp:
    # Create the blackpink.txt file
    with open(txt_file, "w") as txt_fp:
        for i, item in enumerate(list(img_dims.items())):
            key, value = item
            box_w, box_h = value  
            shape_fp.write(f"{box_w} {box_h}")
            txt_fp.write(img_file_prefix+key)

            # Write a new line unless it is the last entry
            if i < len(img_dims)-1:
                shape_fp.write("\n")
                txt_fp.write("\n")

### Create the labels by conversion from coco to Yolo

In [74]:
coco_annots = pd.read_csv('./blackpink.csv', header=0)
coco_annots.head()


,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,bp1.jpg,134310,"{""caption"":"""",""public_domain"":""no"",""image_url""...",4,0,"{""name"":""rect"",""x"":343,""y"":26,""width"":181,""hei...","{""name"":""jennie"",""type"":""human"",""image_quality..."
1,bp1.jpg,134310,"{""caption"":"""",""public_domain"":""no"",""image_url""...",4,1,"{""name"":""rect"",""x"":109,""y"":28,""width"":185,""hei...","{""name"":""jisoo"",""type"":""human"",""image_quality""..."
2,bp1.jpg,134310,"{""caption"":"""",""public_domain"":""no"",""image_url""...",4,2,"{""name"":""rect"",""x"":616,""y"":6,""width"":190,""heig...","{""name"":""rose"",""type"":""human"",""image_quality"":..."
3,bp1.jpg,134310,"{""caption"":"""",""public_domain"":""no"",""image_url""...",4,3,"{""name"":""rect"",""x"":890,""y"":1,""width"":228,""heig...","{""name"":""lisa"",""type"":""human"",""image_quality"":..."
4,bp2.jpg,66729,"{""caption"":"""",""public_domain"":""no"",""image_url""...",4,0,"{""name"":""rect"",""x"":62,""y"":246,""width"":197,""hei...","{""name"":""jennie"",""type"":""human"",""image_quality..."


In [85]:
def convert_to_yolo(box_x, box_y, box_w, box_h, img_w, img_h):
    xmin, ymin, xmax, ymax = box_x, box_y, box_x+box_w, box_y+box_h
    w = float(xmax - xmin)
    h = float(ymax - ymin)
    x = (2*xmax-w)/2.0
    y = (2*ymax-h)/2.0

    return x/img_w, y/img_h, w/img_w, h/img_h

# convert_to_yolo(343, 26, 181, 156, 1200, 496)

label_root = './YoloV3/data/blackpink/labels/'
for imgname in img_dims.keys():
    label_filename = label_root + os.path.splitext(imgname)[0] + '.txt'
    print(label_filename)
    with open(label_filename, "w") as label_fp:
        img_annots_df = coco_annots[coco_annots["filename"] == imgname]
        img_annots_df.reset_index(drop=True, inplace=True)
        for i, row in img_annots_df.iterrows():
            # print(row["region_shape_attributes"])
            img_w, img_h = img_dims[imgname]
            _, x, y, w, h = json.loads(row["region_shape_attributes"]).values()
            c1, c2, c3, c4 = convert_to_yolo(x, y, w, h, img_w, img_h)
            new_line = "\n" if i<len(img_annots_df)-1 else ""
            label_fp.write(f"{row['region_id']} {c1} {c2} {c3} {c4}{new_line}")

# 343 26 181 156
# 109 28 185 186
# 616 6 190 222
# 890 1 228 240

./YoloV3/data/blackpink/labels/bp1.txt
./YoloV3/data/blackpink/labels/bp10.txt
./YoloV3/data/blackpink/labels/bp11.txt
./YoloV3/data/blackpink/labels/bp12.txt
./YoloV3/data/blackpink/labels/bp13.txt
./YoloV3/data/blackpink/labels/bp14.txt
./YoloV3/data/blackpink/labels/bp15.txt
./YoloV3/data/blackpink/labels/bp16.txt
./YoloV3/data/blackpink/labels/bp17.txt
./YoloV3/data/blackpink/labels/bp18.txt
./YoloV3/data/blackpink/labels/bp19.txt
./YoloV3/data/blackpink/labels/bp2.txt
./YoloV3/data/blackpink/labels/bp20.txt
./YoloV3/data/blackpink/labels/bp21.txt
./YoloV3/data/blackpink/labels/bp22.txt
./YoloV3/data/blackpink/labels/bp23.txt
./YoloV3/data/blackpink/labels/bp24.txt
./YoloV3/data/blackpink/labels/bp25.txt
./YoloV3/data/blackpink/labels/bp3.txt
./YoloV3/data/blackpink/labels/bp4.txt
./YoloV3/data/blackpink/labels/bp5.txt
./YoloV3/data/blackpink/labels/bp6.txt
./YoloV3/data/blackpink/labels/bp7.txt
./YoloV3/data/blackpink/labels/bp8.txt
./YoloV3/data/blackpink/labels/bp9.txt


### Printint out content for .shapes file

In [3]:
import cv2 as cv
import glob 

img_dims = dict()
for imgfile in glob.glob('./images/kungfu_panda/*.jpg'):
    # print(imgfile) 
    img = cv.imread(imgfile)
    box_h, box_w, _ = img.shape
    print(box_w, box_h)

452 477
562 371
657 381
355 443
287 307
280 247
192 220
195 245
347 287
173 168
282 333
367 302
180 161
338 362
638 533
312 292
328 358
231 207
295 302
295 251
221 187
377 318
312 280
322 292
182 177
450 501
512 491
678 427
687 450
525 382
543 407
400 400
400 400
400 400
400 400
512 652
250 350
580 388
578 372
378 348
455 386
283 302
333 260
536 400
442 328
472 377
397 250
402 330
401 337
561 451
437 492
367 427
563 457
306 317
447 357
497 598
382 392
291 307
436 458
562 353
172 247
208 272
693 618
317 410
161 187
560 555
398 282
455 542
402 380
426 343
242 380
215 255
376 411
273 443
250 347
400 400
596 506
372 510
346 303
373 426
363 387
337 435
483 387
347 388
462 387
323 443
291 382
295 296
428 310
462 498
491 462
310 330
348 351
545 501
485 368
448 343
271 342
352 345
463 372
405 427
